##### Having a Prior look over existing trained dataset

In [40]:
import pandas as pd
import openpyxl as op

In [41]:
# just to know what is the status of existing dataset.
df=pd.read_excel("training_intent_samples.xlsx")
df.count()

buyer_intent       134
buyer_verbs         76
products           653
seller_intent      511
seller_verbs       200
ad_campaigns        42
negative_verbs      79
intent_suffixes     26
dtype: int64

#### **Training Starts here: txt to tabular data** ---------------- >>

#### *Phase 1 of 2* of Training Data: 
Creating patterns from WORD BANK.

##### Description:
* Features 8
* Label: 9th column
* Dataset name:  dataset_trained.xlsx.xlsx

In [48]:
import re # regular expression for pattern detection
import openpyxl as op # For using tabular data.
from tqdm import tqdm # Special Library for showing Progress.

##### Importing Buyers and Sellers data for training

In [49]:
# Importing Buyers and Sellers data for training
wb=op.load_workbook(r"training_intent_samples.xlsx")
ws=wb.active

buyers= []
buyer_verbs=[]
products=[]
sellers = []
seller_verbs=[]
ad_camp=[]
negations=[]
intent_suffixes=[]

##### Extracting buyers and seller lists of WORD BANK to predict the intents

In [50]:
# Extracting buyers and seller lists of WORD BANK to predict the intents
for row in ws.iter_rows():
    if row[0].value is not None:
        buyers.append(str(row[0].value))
        
    if row[1].value is not None:
        buyer_verbs.append(str(row[1].value))

    if row[2].value is not None:
        products.append(str(row[2].value))
        
    if row[3].value is not None:
        sellers.append(str(row[3].value))
        
    if row[4].value is not None:
        seller_verbs.append(str(row[4].value))
        
    if row[5].value is not None:
        ad_camp.append(str(row[5].value))
        
    if row[6].value is not None:
        negations.append(str(row[6].value))
        
    if row[7].value is not None:
        intent_suffixes.append(str(row[7].value))
        


##### Creating Regex patterns to match later for 4 Classes: 
Buyer, Seller, Buyer Seller and Nothing Intent.

In [51]:
# Creating patterns for regular expression for buyer and seller intents
buyer_pattern = r'\b(?:' + '|'.join(map(re.escape, buyers)) + r')\b'
buyer_verbs_pattern = r'\b(?:' + '|'.join(map(re.escape, buyer_verbs)) + r')\b'
products_pattern = r'\b(?:' + '|'.join(map(re.escape, products)) + r')'
sellers_pattern = r'\b(?:' + '|'.join(map(re.escape, sellers)) + r')\b'
seller_verbs_pattern = r'\b(?:' + '|'.join(map(re.escape, seller_verbs)) + r')\b'
ad_camp_pattern = r'\b(?:' + '|'.join(map(re.escape, ad_camp)) + r')\b'
negations_pattern = r'\b(?:' + '|'.join(map(re.escape, negations)) + r')\b' 
intent_suffixes_pattern = r'\b(?:' + '|'.join(map(re.escape, intent_suffixes)) + r')' 

wb.close()

##### Reading Training Data : Raw Sentences
There are options to update data set:
1. Using individual sentences, and
2. Using one or more .txt files having many sentences.

In [52]:
# Reading training Data
# Choice 
choice=int(input('''For sentence input select "1" 
For txt(s) files select "2" 
>>> ''').strip())


#### *Phase 2 of 2* of Training Data:
Creating a function evaluate( line, label ): 
For processing each sentences one by one.

In [53]:
def evaluate(line,label):
    text=line
    
    # setting row back to 0
    row=0 
    wb2=op.load_workbook(r"G:\GitClone\NLP\NLP\dataset_trained.xlsx.xlsx")
    ws2 = wb2.active
    # Adding Header to all column in trained dataset
    if ws2.cell(row+1,1).value!="buyer":
        ws2.cell(row+1,1).value="buyer"
        ws2.cell(row+1,2).value="seller"
        ws2.cell(row+1,3).value="seller_verb"
        ws2.cell(row+1,4).value="buyer_verb"
        ws2.cell(row+1,5).value="products"
        ws2.cell(row+1,6).value="ad_campaign"
        ws2.cell(row+1,7).value="intent_sufixes"
        ws2.cell(row+1,8).value="negations"
        ws2.cell(row+1,9).value="label"
              
    row= ws2.max_row # setting row to next empty cell
    
    # Finding all occurrences using the re module for patterns
    buyer_match = re.findall(buyer_pattern, text, flags=re.IGNORECASE)
    buyer_verbs_match = re.findall(buyer_verbs_pattern, text, flags=re.IGNORECASE)
    products_match = re.findall(products_pattern, text, flags=re.IGNORECASE)
    seller_match = re.findall(sellers_pattern, text, flags=re.IGNORECASE)
    seller_verbs_match = re.findall(seller_verbs_pattern, text, flags=re.IGNORECASE)
    ad_camp_match = re.findall(ad_camp_pattern, text, flags=re.IGNORECASE)
    negations_match = re.findall(negations_pattern, text, flags=re.IGNORECASE)
    intent_suffixes_match= re.findall(intent_suffixes_pattern, text, flags=re.IGNORECASE)

    B=len(buyer_match)
    S=len(seller_match)
    BV=len(buyer_verbs_match)
    SV=len(seller_verbs_match)
    P=len(products_match)
    Ad=len(ad_camp_match)
    IS=len(intent_suffixes_match)
    PN=len(negations_match)
    
    ws2.cell(row+1,1).value=B
    ws2.cell(row+1,2).value=S
    ws2.cell(row+1,3).value=SV
    ws2.cell(row+1,4).value=BV
    ws2.cell(row+1,5).value=P
    ws2.cell(row+1,6).value=Ad 
    ws2.cell(row+1,7).value=IS
    ws2.cell(row+1,8).value=PN


    # adding label to the training data
    ws2.cell(row+1,9).value=label.upper()

    row+=1
    wb2.save(r"G:\GitClone\NLP\NLP\dataset_trained.xlsx.xlsx")

##### Interacting with User for raw data feed: 
User has to provide either _INDIVIDUAL SENTENCE_ or _.txt FILE(s)_ with their repective Labels: B, S, BS or N.

In [54]:
training_files_n_labels=[] # for taking number of file's paths and their respective labels

if choice==1:
    training_file_txt=input("Waiting for your sentence>>>")
    label=input("Select Labels: B, S, BS or N >>>"  ).strip()
    evaluate(training_file_txt,label)
    print("Completed 🟢")
    
    
elif choice==2:
    num=int(input("Number of files >>>  ").strip())
    
    for c in range(num):
        path=input(f"File path #{c+1} >>>").strip()
        label=input("Select Labels:  B, S, BS or N >>>").strip().upper()
        training_files_n_labels.append([path,label])
        
    for f in range(num): # num is number of files that user feeded for training
        training_file_txt=training_files_n_labels[f][0]
        label=training_files_n_labels[f][1]
        
        with open(training_file_txt, 'r') as file: # just to find total length for progress bar
            length=len([line for line in file])
                
                        
        with open(training_file_txt, 'r') as file:
            progress_bar = tqdm(total=length, unit="iteration") # progress Bar
            for line in file:
                progress_bar.update(1)
                evaluate(line.strip(),label)
                
            progress_bar.close()
                
    print("Completed 🟢")

100%|██████████| 500/500 [03:15<00:00,  2.55iteration/s]

Completed 🟢


#### Training Part Ends Here. Now Trained Dataset is ready to be used. I has encoded features for sentence's description. --------<<>>